In [1]:
import os,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath("InjecAgent")) 
sys.path.append(os.path.abspath("aside"))
from aside.model import *
from aside.model_api import *
import torch
import json
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from utils.utils import *
from utils.model_utils import load_model
from utils.plot_utils import *
torch.set_grad_enabled(False)

seed_all()
from vllm import LLM, SamplingParams
from functools import partial

from agentdojo.attacks.attack_registry import ATTACKS, load_attack
from agentdojo.task_suite.load_suites import get_suite, get_suites
from agentdojo.task_suite.task_suite import *
from agentdojo.agent_pipeline.agent_pipeline import DEFENSES, AgentPipeline, PipelineConfig



/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: librt.so.1, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: libpthread.so.0, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: libstdc++.so.6, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: libm.so.6, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/minic

INFO 10-01 15:14:17 [__init__.py:216] Automatically detected platform cuda.


# Load Trained Model

In [ ]:
device = 'cuda:0'
data_dir = '/dataset/common/huggingface/model'
torch_dtype = torch.bfloat16
max_ctx_len = 32768
# model_path = 'Qwen/Qwen3-8B'
model_path = os.path.join(data_dir,'Qwen3-8B-Tool_Vanilla_SFT')

use_vllm=True

model,tokenizer,is_aside = load_model(model_path,use_vllm=use_vllm,dtype=torch_dtype,vllm_kwargs = {'gpu_memory_utilization':0.8,'enable_chunked_prefill':False},max_ctx_len=max_ctx_len)

if is_aside:
    use_vllm = False
    
def assign_segment_ids(encoded,tokenizer):
    segment_ids = multiturn_aside_encode(encoded,tokenizer,until_last_token = "\n<|im_start|>assistant") # allow the last assistant to be unclosed
    encoded['segment_ids'] = segment_ids
    # segs = get_aside_segments(encoded,tokenizer)
    # pprint (f'PROMPT: {tokenizer.decode(encoded["input_ids"][0])}')
    # for i, seg in enumerate(segs):
    #     pprint (f'SEG {i} {seg}')
    return encoded
additional_encode_fn = partial(assign_segment_ids,tokenizer=tokenizer) if is_aside else lambda x:x
is_api = any([s in model_path.lower() for s in ['gpt','deepseek','claude']])

INFO 10-01 09:42:23 [utils.py:328] non-default args: {'max_model_len': 32768, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'enable_chunked_prefill': False, 'model': '/dataset/common/huggingface/model/Qwen3-8B-Tool_Vanilla_SFT'}
INFO 10-01 09:42:35 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-01 09:42:35 [__init__.py:1815] Using max model len 32768
INFO 10-01 09:42:37 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 10-01 09:42:39 [__init__.py:2974] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-01 09:42:46 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=2376141) INFO 10-01 09:42:47 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=2376141) INFO 10-01 09:42:47 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='/dataset/common/huggingface/model/Qwen3-8B-Tool_Vanilla_SFT', speculative_config=None, tokenizer='/dataset/common/huggingface/model/Qwen3-8B-Tool_Vanilla_SFT', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, tr

[W1001 09:42:50.994397287 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=2376141) INFO 10-01 09:42:50 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=2376141) WARNING 10-01 09:42:50 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=2376141) INFO 10-01 09:42:50 [gpu_model_ru

Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  14% Completed | 1/7 [00:05<00:33,  5.53s/it]
Loading safetensors checkpoint shards:  29% Completed | 2/7 [00:11<00:28,  5.65s/it]
Loading safetensors checkpoint shards:  43% Completed | 3/7 [00:18<00:25,  6.30s/it]
Loading safetensors checkpoint shards:  57% Completed | 4/7 [00:24<00:18,  6.17s/it]
Loading safetensors checkpoint shards:  71% Completed | 5/7 [00:28<00:10,  5.48s/it]
Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:34<00:05,  5.69s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:40<00:00,  5.80s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:40<00:00,  5.81s/it]
(EngineCore_DP0 pid=2376141) 


(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:32 [default_loader.py:268] Loading weights took 41.16 seconds
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:33 [gpu_model_runner.py:2392] Model loading took 15.2683 GiB and 41.466055 seconds
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:41 [backends.py:539] Using cache directory: /export/home2/weijie210/.cache/vllm/torch_compile_cache/9ac66a0ca9/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:41 [backends.py:550] Dynamo bytecode transform time: 7.87 s
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:43 [backends.py:194] Cache the graph for dynamic shape for later use
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:49 [backends.py:215] Compiling a graph for dynamic shape takes 6.97 s
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:50 [monitor.py:34] torch.compile takes 14.84 s in total
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:52 [gpu_worker.py:298] Available KV cache memory: 46.66 GiB
(EngineCore_DP0 pi

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|█| 67/67 [00:03<0


(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:57 [gpu_model_runner.py:3118] Graph capturing finished in 5 secs, took 0.62 GiB
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:57 [gpu_worker.py:391] Free memory on device (78.76/79.25 GiB) on startup. Desired GPU memory utilization is (0.8, 63.4 GiB). Actual usage is 15.27 GiB for weight, 1.45 GiB for peak activation, 0.02 GiB for non-torch memory, and 0.62 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=49284854784` to fit into requested memory, or `--kv-cache-memory=65780986880` to fully utilize gpu memory. Current kv cache memory in use is 50104841216 bytes.
(EngineCore_DP0 pid=2376141) INFO 10-01 09:43:57 [core.py:218] init engine (profile, create kv cache, warmup model) took 23.93 seconds
INFO 10-01 09:43:58 [llm.py:295] Supported_tasks: ['generate']
INFO 10-01 09:43:58 [__init__.py:36] No IOProcessor plugins requested by the model


ERROR 10-01 11:28:51 [core_client.py:564] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.


In [5]:
def tool_prompt_format(messages,tools,tokenizer,encode=False,enable_thinking=False): # for tool-use agents like Qwen
    if messages[-1]['role'] == 'assistant':
        add_generation_prompt = False
    else:
        add_generation_prompt = True
    if messages[0]['role'] == 'system':
        messages = messages[1:] # remove system prompt for tool-use models
    if 'qwen' in tokenizer.name_or_path.lower():
        formatted = tokenizer.apply_chat_template(messages,add_generation_prompt=add_generation_prompt,tokenize=False,enable_thinking=enable_thinking,tools=tools)
    else:
        formatted = tokenizer.apply_chat_template(messages,add_generation_prompt=add_generation_prompt,tokenize=False,tools=tools)
    # pprint (formatted)
    return encode_fn(formatted,tokenizer) if encode else formatted

In [7]:
defense = None
if not is_api:
    # Tool LLM
    agent_pipeline = AgentPipeline.from_config(
            PipelineConfig(
                llm='local-tool',
                defense=defense,
                system_message_name=None,
                system_message=None,
                tool_output_format=None,
                extra_args={
                    'model':model,
                    'tokenizer':tokenizer,
                    'encode_fn': partial(tool_prompt_format,tokenizer=tokenizer),
                    'additional_encode_fn': additional_encode_fn,
                    'gen_kwargs': {'max_new_tokens':512,'do_sample':False,'eos_token_id':[tokenizer.eos_token_id]} if not use_vllm else SamplingParams(max_tokens=512,temperature=0,stop_token_ids = [tokenizer.eos_token_id]),
                    'is_vllm': use_vllm,
                },
                # max_iters=3,
            )
        )
else:
    with open(f'/export/home2/weijie210/ipi_huawei/ds_key.txt','r') as f:
        ds_key = f.read().strip()
    os.environ['OPENAI_API_KEY'] = ds_key
    agent_pipeline = AgentPipeline.from_config(
            PipelineConfig(
                llm=model_path,
                defense=defense,
                system_message_name=None,
                system_message=None,
                tool_output_format=None,
                extra_args ={'max_tokens':512}
                
            )
        )

# Function for Evaluation

In [8]:
from agentdojo.base_tasks import BaseInjectionTask, BaseUserTask

def run_task(suite,user_task,injection_task=None,task_injection = {},return_messages=False):
    environment = suite.load_and_inject_default_environment(task_injection)
    if isinstance(user_task, BaseUserTask):
        task_environment = user_task.init_environment(environment)
    else:
        task_environment = environment
    pre_environment = task_environment.model_copy(deep=True)
    if isinstance(user_task, BaseUserTask):
        prompt = user_task.PROMPT
    else:
        prompt = user_task.GOAL

    messages = []
    runtime = FunctionsRuntime(suite.tools)

    _, _, task_environment, messages, _ = agent_pipeline.query(prompt, runtime, task_environment)
    model_output = model_output_from_messages(messages)
    functions_stack_trace = functions_stack_trace_from_messages(messages)
    utility = suite._check_task_result(
            user_task,
            model_output or [],
            pre_environment,
            task_environment,  # type: ignore
            functions_stack_trace,
        )
    security = None
    if injection_task is not None:
        security = suite._check_task_result(
            injection_task,
            model_output or [],
            pre_environment,
            task_environment,  # type: ignore
            functions_stack_trace,
        )
    if not return_messages:
        return utility,security
    else:
        return utility,security, messages
    
def count_turns(messages):
    ass_count = 0
    has_user = False
    for msg in messages[2:]: # skip system and first user
        if msg['role'] == 'assistant':
            ass_count += 1
        elif msg['role'] == 'user':
            has_user = True
    return {'turn':ass_count,'additional_user':has_user}


# Eval

In [ ]:
benchmark_version = 'v1.2.1'
suites = tuple(get_suites(benchmark_version).keys())
suites = ['slack','banking','travel','workspace']
# suites = ['slack']
attack_name = 'important_instructions'

all_suite_results = defaultdict(dict)
all_suite_asr_messages = defaultdict(dict)
all_suite_stats = defaultdict(dict)
debug_messages = defaultdict(list)

if 'qwen' in model_path.lower():
    base_model_name = 'qwen3'
elif 'llama3' in model_path.lower():
    base_model_name = 'llama3'
elif 'gemma' in model_path.lower():
    base_model_name = 'gemma2'
else:
    base_model_name = model_path


model_n = model_path.split('/')[-1]
result_dict = 'results/agentdojo'
os.makedirs(result_dict,exist_ok=True)

result_file = f'{result_dict}/{base_model_name}_{model_n}.json'
msg_file = f'{result_dict}/{base_model_name}_{model_n}_messages.pkl'

if not os.path.exists(result_file):
    for suite_name in suites:
        suite = get_suite(benchmark_version, suite_name)
        print(f"**Running benchmark for suite: '{suite.name}'**")
        user_tasks_to_run = suite.user_tasks.values()
        print (f"Num User tasks to run: {len(user_tasks_to_run)}")
        attacker_ = load_attack(attack_name, suite, target_pipeline = agent_pipeline)
        print ('is dos_attack:', attacker_.is_dos_attack)
        injection_tasks_to_run = suite.injection_tasks
        print (f"Num Injection tasks to run: {len(injection_tasks_to_run)}")

        #Injection utility
        injection_task_utility = {}
        injection_turn_count = {}
        for injection_task_id, injection_task in tqdm(injection_tasks_to_run.items(),total = len(injection_tasks_to_run),desc = f'Running Injection utility for {suite_name}'):
            utility,_,message = run_task(suite,injection_task,return_messages=True)
            injection_task_utility[injection_task_id] = utility
            debug_messages['injection'].append(message)
            injection_turn_count[injection_task_id] = count_turns(message)
            
            
        print (f'{suite_name} Injection utility: {np.mean(list(injection_task_utility.values())):.2f}')
        all_suite_results[suite_name]['injection_utility'] = list(injection_task_utility.values())
        all_suite_stats['injection'][suite_name] = injection_turn_count

        ## Non-injection utility
        task_utility = {}
        task_turn_count = {}
        for user_task in tqdm(user_tasks_to_run,total = len(user_tasks_to_run),desc = f'Running Task utility for {suite_name}'):
            utility,_,message = run_task(suite,user_task,return_messages=True)
            task_utility[user_task.ID] = utility
            debug_messages['utility'].append(message)
            task_turn_count[user_task.ID] = count_turns(message)
        print (f'{suite_name} Task utility: {np.mean(list(task_utility.values())):.2f}')
        all_suite_results[suite_name]['task_utility'] = list(task_utility.values())
        all_suite_stats['task'][suite_name] = task_turn_count

        ## With attack
        utility_with_attack = {}
        all_messages = {}
        suite_security_results =  {}
        for user_task in tqdm(user_tasks_to_run,total = len(user_tasks_to_run),desc = f'Running Attack for {suite_name}'):
            for injection_task_id in injection_tasks_to_run:
                injection_task = suite.get_injection_task_by_id(injection_task_id)
                task_injections = attacker_.attack(user_task, injection_task)
                utility,security,messages = run_task(suite,user_task,injection_task,task_injections,return_messages=True)
                utility_with_attack[(user_task.ID, injection_task_id)] = utility
                suite_security_results[(user_task.ID, injection_task_id)] = security
                all_messages[(user_task.ID, injection_task_id)] = messages
        all_suite_asr_messages[suite_name] = all_messages
        all_suite_results[suite_name]['asr'] = list(suite_security_results.values())
        all_suite_results[suite_name]['attack_utility'] = list(utility_with_attack.values())

        print (f'{suite_name} Task with injection Utility: {np.mean(list(utility_with_attack.values())):.2f}')
        print (f'{suite_name} ASR: {np.mean(list(suite_security_results.values())):.2f}')
    
    all_suite_results_dict = {'results':all_suite_results,'stats':all_suite_stats}
    with open(result_file, 'w') as f:
        json.dump({'results':all_suite_results,'stats':all_suite_stats}, f, indent=4)
        

else:
    with open(result_file, 'r') as f:
        all_suite_results_dict = json.load(f)

for suite_name in all_suite_results_dict['results']:
    suite_results = all_suite_results_dict['results'][suite_name]
    print (f"Results for {suite_name}:")
    print (f"Injection utility: {np.mean(suite_results['injection_utility']):.2f}")
    print (f"Task utility: {np.mean(suite_results['task_utility']):.2f}")
    print (f"Task with injection utility: {np.mean(suite_results['attack_utility']):.2f}")
    print (f"ASR: {np.mean(suite_results['asr']):.2f}")

Results for slack:
Injection utility: 1.00
Task utility: 0.48
Task with injection utility: 0.34
ASR: 0.16
Results for banking:
Injection utility: 0.89
Task utility: 0.38
Task with injection utility: 0.32
ASR: 0.20
Results for travel:
Injection utility: 0.86
Task utility: 0.30
Task with injection utility: 0.25
ASR: 0.00
Results for workspace:
Injection utility: 0.43
Task utility: 0.30
Task with injection utility: 0.27
ASR: 0.00


[rank0]:[W1001 11:28:51.177794964 ProcessGroupNCCL.cpp:1538] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [ ]:
result_stats= all_suite_results_dict['stats']
turns = []
utility = []
for task_type in result_stats: # compute correlation of turn counts and utility
    for suite_name in result_stats[task_type]:
        task_stats = result_stats[task_type][suite_name]
        for task_id in task_stats:
            turns.append(task_stats[task_id]['turn'])
            if task_type == 'task':
                utility.append(all_suite_results_dict['results'][suite_name]['task_utility'])
            elif task_type == 'injection':
                utility.append(all_suite_results_dict['results'][suite_name]['injection_utility'])
    

In [ ]:
count = 0
for i,s in enumerate(injection_task_utility.values()):
    if not s:
        if count == 0:
            print (len(debug_messages['injection'][i]))
            for m in debug_messages['injection'][i]:
                pprint (str(m))
            break
        count += 1


# count = 0
# for k,s in suite_security_results.items():
#     if not s and utility_with_attack[k]:
#         if count == 0:
#             print (len(all_messages[k]))
#             for m in all_messages[k]:
#                 pprint (str(m))
#             break
#         count += 1

In [ ]:
for suite_name in all_suite_results:
    suite_results = all_suite_results[suite_name]
    print (f"Results for {suite_name}:")
    print (f"Injection utility: {suite_results['injection_utility']:.2f}")
    print (f"Task utility: {suite_results['task_utility']:.2f}")
    print (f"Task with injection utility: {suite_results['attack_utility']:.2f}")
    print (f"ASR: {suite_results['asr']:.2f}")

# Check result

In [6]:
# base_model_name = 'qwen3'
base_model_name = 'qwen3'
result_dict = 'results/agentdojo'
# model_n = 'ToolACE-2.5-Llama-3.1-8B'
# model_n = 'Qwen3-8B'
model_n = 'Qwen3-8B'
with open(f'{result_dict}/{base_model_name}_{model_n}.json', 'r') as f:
    all_suite_results = json.load(f)


for suite_name in all_suite_results:
    suite_results = all_suite_results[suite_name]
    print (f"** {suite_name} **")
    print (f"Injec U: {np.mean(suite_results['injection_utility']):.2f}")
    print (f"Task U: {np.mean(suite_results['task_utility']):.2f}")
    print (f"Task/Injec U: {np.mean(suite_results['attack_utility']):.2f}")
    print (f"ASR: {np.mean(suite_results['asr']):.2f}")
    print ('--'*20)

** slack **
Injec U: 1.00
Task U: 0.76
Task/Injec U: 0.40
ASR: 0.46
----------------------------------------
** banking **
Injec U: 0.89
Task U: 0.44
Task/Injec U: 0.27
ASR: 0.26
----------------------------------------
** travel **
Injec U: 1.00
Task U: 0.15
Task/Injec U: 0.14
ASR: 0.28
----------------------------------------
** workspace **
Injec U: 0.36
Task U: 0.30
Task/Injec U: 0.34
ASR: 0.01
----------------------------------------
